<a href="https://colab.research.google.com/github/tantowijh/colab-collection/blob/main/Downloader_to_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Mount Google Drive**
Run the following command to mount google drive:

In [ ]:
#@title **[1] Link your Google Drive**
#@markdown <center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#@title **[2] Run this code if you want a longer run time (Optional but recommended)</font>**
#@markdown *This cell runs a JS code that automatically reconnects you to avoid inactivity due to idle.*

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function connectRefresher() {
       window.ConnectButtonIntervalId = setInterval(function ConnectButton(){
                console.log("connected"); 
                document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
                document.querySelector("colab-sessions-dialog").shadowRoot.querySelector("#footer > div > paper-button").click();
                console.log("closed the dialog!!"); 
            },60000);
    }
    
 function clearRefresher() { 
           console.log("clear Interval called !!");
           clearInterval(window.ConnectButtonIntervalId);
    }

 connectRefresher(); //to connect the refresher
 clearRefresher(); //to disconnect the refresher
'''))

clear_output()

## **Google Drive Downloader**
Run the following command to download files into google drive:

In [ ]:
#@title **[1] Download From Google Drive**
#@markdown *Input file name that you want to **save as** and consider adding an **extension file type** at the end, then put the URL and download location*.

import re, os
from IPython.display import clear_output

LargeFile = True #@param {type:"boolean"}
FileNametoSave = "_Kusonime__Pikiran_Busuk_BD_720P.rar" #@param {type:"string"}
GoogleSharedLink = "https://drive.google.com/uc?export=download&id=12ANcXVCYFzbIJ45_zeSV7K0hn68fkoPf" #@param {type:"string"}
# Match six-letter word
pattern = r"[-\w]{18,}"
# match() method
result = re.match(pattern, GoogleSharedLink)
# Output None
# search() method
result = re.search(pattern, GoogleSharedLink)
fileId = result.group()
# findall() method
result = re.findall(pattern, GoogleSharedLink)

# Define download location
DownloadLocation = "/content/drive/Shareddrives/01. Anime Server/01. Anime Series/Kuzu no Honkai" #@param {type:"string"}

from pathlib import Path
Path(DownloadLocation).mkdir(parents=True, exist_ok=True)

%cd '{DownloadLocation}'
clear_output()

if LargeFile == False:
  !wget -q --show-progress --progress=bar:force --no-check-certificate 'https://docs.google.com/uc?export=download&id={fileId}' -O '{FileNametoSave}'
else:
  !wget -q --show-progress --progress=bar:force --load-cookies '/tmp/cookies.txt' "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate "https://docs.google.com/uc?export=download&id={fileId}" -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={fileId}" -O '{FileNametoSave}' && rm -rf /tmp/cookies.txt



In [ ]:
# @title **[2] Download from Git**

import os
import subprocess

#@markdown *Put file download location*
Download_Location = "/content/drive/MyDrive/Subtitle" #@param {type:"string"}
Download_Link = "https://github.com/MatheusGrilo/OpenSubs.git" #@param {type:"string"}

%cd '{Download_Location}' 
!git clone '{Download_Link}' 

In [ ]:
from sys import path
#@title **[3] Direct File download**
#@markdown *The files needs to be of some kind of extension or something similar, you get it right? e.x. .exe .zip .apk etc. The link should be - https://*

DownloadMethod = "ARIA2" #@param ["WGET", "ARIA2"]
DownloadOption = "Single Download" #@param ["Single Download", "Multiple Download"]
#@markdown *Provide txt file path for multiple download option, separate download link in new line. Example: content/drive/MyDrive/downloadlink.txt*

import os
import subprocess

from os import path
from IPython.utils import io

if not path.exists("/usr/local/bin/aria2c"):
  with io.capture_output() as captured:
    !sudo apt install aria2

Download_Location = "/content" #@param {type:"string"}
Download_Link = "" #@param {type:"string"}

def wget():
  def SingleDwnld():
    %cd '{Download_Location}'
    !wget '{Download_Link}'

  def MultipleDwnld():
    %cd '{Download_Location}'
    !wget -i '{Download_Link}'

  if DownloadOption == "Single Download":
    SingleDwnld()
  if DownloadOption == "Multiple Download":
    MultipleDwnld()
def aria2():
  def SingleDwnld():
    %cd '{Download_Location}'
    !aria2c '{Download_Link}'

  def MultipleDwnld():
    %cd '{Download_Location}'
    !aria2c -i '{Download_Link}' --file-allocation=none -c --auto-file-renaming=false

  if DownloadOption == "Single Download":
    SingleDwnld()
  if DownloadOption == "Multiple Download":
    MultipleDwnld()
if DownloadMethod == "WGET":
  wget()
if DownloadMethod == "ARIA2":
  aria2()


In [ ]:
#@title **[4] Direct File download and Zip extraction**
#@markdown *Use this if you want to download a zip file and extract it on the same google drive.*

import os
import subprocess

Download_Location = "/content/drive/MyDrive/Subtitle" #@param {type:"string"}
Download_Link = "https://www.ffonts.net/AR-CENA.font.zip" #@param {type:"string"}
ExtractedFolderName = "Extracted" #@param {type:"string"}

%cd '{Download_Location}'
!wget '{Download_Link}'

#@markdown *Make sure there is no other zip file. Or will it be extracted too.* 

!unzip *.zip -d {ExtractedFolderName}
%mv '{Download_Location}/*/*' '{Download_Location}/{ExtractedFolderName}/ -n'

In [ ]:
#@title **[5] Zippyshare Downloader**
#@markdown *Start downloading from zippyshare*

import os
import subprocess

Download_Location = "/content/drive/Shareddrives/01. Anime Server/01. TV Show/30. Kanojo, Okarishimasu/Season 2" #@param {type:"string"}
Download_Link = "https://www67.zippyshare.com/d/M589DIuG/291/%5bLENDRIVE%5d_KanoKari_S2_-_06_%5b1080p_HEVC_10bit%5d%5bDualsubs%5d.mkv" #@param {type:"string"}
Download_Referrer = "https://www53.zippyshare.com/v/M589DIuG/file.html" #@param {type:"string"}
JSESSION_ID = "BF70FF2AB9A828F49CB8A58268AF6431" #@param {type:"string"}

%cd '{Download_Location}' 
!wget '{Download_Link}' --referer='{Download_Referrer}' --cookies=off --header "Cookie: JSESSIONID={JSESSION_ID}" --user-agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'

In [ ]:
#@title **[6] Hosting Site File Download**
#@markdown *Example: Zippyshare and One Drive*

import os
import subprocess

Download_Location = "/content/drive/Shareddrives/01. Anime Server/01. Anime/01. TV Show/37. Hoshi no Samidare/Season 1" #@param {type:"string"}
Download_Link = "https://download1334.mediafire.com/sj6wpvgcxhpg/y9m6neiiybtx2qr/%5BLENDRIVE%5D_Hoshi_no_Samidare_-_04_%5B1080p_HEVC_10bit%5D%5BDualsubs%5D.mkv" #@param {type:"string"}
Download_Referrer = "https://www.mediafire.com/file/y9m6neiiybtx2qr/%255BLENDRIVE%255D_Hoshi_no_Samidare_-_04_%255B1080p_HEVC_10bit%255D%255BDualsubs%255D.mkv/file" #@param {type:"string"}
Cookie_Name = "" #@param {type:"string"}
Cookie_Auth = "" #@param {type:"string"}

%cd '{Download_Location}' 
!wget '{Download_Link}' --referer='{Download_Referrer}' --cookies=off --header "Cookie: {Cookie_Name}={Cookie_Auth}" --user-agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'

##<left><img src="https://ar.toneden.io/13766685/6070f58d-091b-4a80-bd5c-24378e308116" height="50" alt="Zippyshare logo"/></left>
## ___**Zippyshare Downloader**___

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <center><img src="https://ar.toneden.io/13766685/6070f58d-091b-4a80-bd5c-24378e308116" height="100" alt="Zippyshare logo"/></center>
#@markdown <center><h3>Remote upload from Zippyshare to Google drive</h3></center><br>


from IPython.utils import io
from IPython.display import HTML, clear_output
with io.capture_output() as captured:
  %cd /content
  !git clone https://github.com/mcrapet/plowshare.git
  %cd plowshare
  !make install
  !plowmod --install
  !apt-get install nodejs
  !cd ..


clear_output()

print("Server Started Successfully")


Server Started Successfully


In [ ]:
#@markdown <h3>⬅️ Click Here to START Download</h3>

import os
direct_URL = "https://www18.zippyshare.com/d/JN8CMwu5/507320/%5bLENDRIVE%5d_Tensei_Kenja_-_08_%5b1080p_HEVC_10bit%5d%5bDualsubs%5d.mkv" #@param {type:"string"}
output_dir = "/content" #@param {type:"string"}

!plowdown "$direct_URL" -o "$output_dir"
from IPython.utils import io
with io.capture_output() as captured:
  %cd /content/

In [ ]:
#@markdown <h3>⬅️ Click Here to START Batch Download</h3>
#@markdown <h5> Make "download.txt" file contaning batch links of zippyshare and upload here to down them all.

%cd /content/plowshare

!rm *.txt

from google.colab import files
files.upload()

import os
output_dir = "/content/drive/My Drive" #@param {type:"string"}

!plowdown "download.txt" -m -o "$output_dir"

%cd /content/





## **Download a Website**
Run the following command to download the website recursively:

```
wget -r --no-parent http://www.example.com
```

This will download the pages without altering their HTML source code.

When you want to change the links on the pages automatically to point to the downloaded files then use this command instead:

```
wget -r --convert-links --no-parent http://www.example.com
```

When all HTML files shall get a .html extension, then add the "--html-extension" option.

```
wget -r --convert-links --html-extension --no-parent http://www.example.com
```

In [ ]:
#@title **[3] Direct File download**
#@markdown *Download website option:*

import os
import subprocess

ChooseOption = "3. Download the pages and change the links on the pages automatically to point to the downloaded files with HTML Extension." #@param ["1. Download the pages without altering their HTML source code.", "2. Download the pages and change the links on the pages automatically to point to the downloaded files.", "3. Download the pages and change the links on the pages automatically to point to the downloaded files with HTML Extension."]

def option1():
  Download_Location = input("Enter location you want to download the website files: ")
  Download_Link = input("Enter website link you want to download: ")
  %cd '{Download_Location}'
  !wget -r --no-parent '{Download_Link}'
  
def option2():
  Download_Location = input("Enter location you want to download the website files: ")
  Download_Link = input("Enter website link you want to download: ")
  %cd '{Download_Location}'
  !wget -r --convert-links --no-parent '{Download_Link}'

def option3():
  Download_Location = input("Enter location you want to download the website files: ")
  Download_Link = input("Enter website link you want to download: ")
  %cd '{Download_Location}'
  !wget -r --convert-links --html-extension --no-parent '{Download_Link}'

if ChooseOption == "1. Download the pages without altering their HTML source code.":
  option1()
if ChooseOption == "2. Download the pages and change the links on the pages automatically to point to the downloaded files.":
  option2()
if ChooseOption == "3. Download the pages and change the links on the pages automatically to point to the downloaded files with HTML Extension.":
  option3()